In [ ]:
!python module_preprocessing.py --default=True --train_ann='/input/train_annotations_equal.json' --dataset_type=1 --test_ann='/input/test_annotations_equal.json' --train_split=0.05

Detectron2 using cocolike structure training mask rcnn 50 layer.

data: module_preprocessing.py
data augmentations: https://jss367.github.io/Data-Augmentation-with-Detectron2.html / https://detectron2.readthedocs.io/modules/data_transforms.html
- flip horisontal 50% prob
- flip vertical 50% prob
- random rotation -20 to 20%. 
- random lightning 0.1 standard deviations. 

3 Stages:
- 1st stage  256x256 images
- 2nd stage  512x512 images
- 3rd stage  756x756 images
- hopefully with variable optimized learning rate. 

Configurations have been changed from the default. Current notebook contains only the mask training part. Documentation for detectron2 documentation: https://detectron2.readthedocs.io/modules/config.html


Metrics: tensorboard (on local: http://127.0.0.1:6006) -> call from another script/notebook.

Submission: module_submittion.py

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

In [ ]:
import os
import copy
import json
import pycocotools
import random 

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #Truncated image -> https://github.com/keras-team/keras/issues/5475

In [ ]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train_v2",{},PATH + "/input/train_annotations_equal.json",PATH + "/input/train_v2/")
register_coco_instances("my_dataset_val_v2",{},PATH + "/input/test_annotations_equal.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train_v2")
dataset_dicts = DatasetCatalog.get("my_dataset_train_v2")

In [ ]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    # List of transforms https://detectron2.readthedocs.io/modules/data_transforms.html
    # Add saturation, add shear orsmth.
    transform_list = [
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                      T.RandomLighting(0.1),
                      T.RandomRotation((-0.2,0.2))
                     ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

In [ ]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [ ]:
#Call from anywhere else. 
#!tensorboard --logdir=run_equal --host=0.0.0.0
#http://0.0.0.0:6006/#scalars

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 20000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only has one class (ship)
cfg.MAX_SIZE_TRAIN = 256 #Max image size 
cfg.OUTPUT_DIR = "./runs/run_50_anchortest"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True # Teach only ships & background
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR" #avoid getting stuck in local minima.
cfg.CUDNN_BENCHMARK = True
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]]
cfg.SOLVER.AMP.ENABLED = True  # Automatic Mixed Precision

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 12
cfg.SOLVER.BASE_LR = 0.001  
cfg.SOLVER.MAX_ITER = 30000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 512 #Max image size 
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR" #avoid getting stuck in local minima. 
cfg.OUTPUT_DIR = "./runs/run_50_anchortest"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]]
cfg.SOLVER.AMP.ENABLED = True  # Automatic Mixed Precision

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 10
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 110000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 756 #Max image size 
cfg.SOLVER.STEPS=(70000, 105000) #reduce gradually lr. 
cfg.OUTPUT_DIR = "./runs/run_50_anchortest"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]]
cfg.SOLVER.AMP.ENABLED = True  # Automatic Mixed Precision

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

In [ ]:
# Create a submission for kaggle. 

# There will be an overload error: https://github.com/pytorch/vision/pull/2705
# With classifier predictions included
!python module_submit.py --model_path="runs/run_50_anchortest" --submit_csv="submit_50_anchortest_1.csv" --score_thres=0.8 --ship_proba_csv="test_ship_proba.csv" --anchor_sizes="small"
# By itself
!python module_submit.py --model_path="runs/run_50_anchortest" --submit_csv="submit_50_anchortest_2.csv" --score_thres=0.8 --anchor_sizes="small"